In [1]:
import os
%pwd

'e:\\datascience\\text\\TextSummarizer\\research'

In [2]:
os.chdir("../")
%pwd

'e:\\datascience\\text\\TextSummarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_Path:Path
    model_ckpt:Path
    num_train_epochs:int
    warmup_steps:int
    per_device_train_batch_size:int
    weight_decay:float
    logging_steps:int
    evaluation_strategy:str
    eval_steps:int
    save_steps:float
    gradient_accumulation_steps:int

In [4]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_path = CONFIG_FILE_PATH,
                 params_path =PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params =read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguements

        create_directories([config.root_dir])
        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            data_Path=config.data_Path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config


        




In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

e:\datascience\text\TextSummarizer\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config

    def train(self):
          device = "cuda" if torch.cuda.is_available() else "cpu"
          tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
          model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
          seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading the data
          dataset_samsum_pt = load_from_disk(self.config.data_path)

          trainer_args = TrainingArguements(
          output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
          per_device_train_batch_size=1, per_device_eval_batch_size=1,
          weight_decay=0.01, logging_steps=10,
          evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
          gradient_accumulation_steps=16
        ) 
          trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])
        
          trainer.train()

        ## Save model
          model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
          tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

^C


In [11]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config = model_trainer_config)
model_trainer.train()


[2025-07-16 18:41:06,969: INFO:common:loaded successfully config\config.yaml file]
[2025-07-16 18:41:06,975: INFO:common:loaded successfully params.yaml file]
[2025-07-16 18:41:06,990: INFO:common:created directory at:artifacts]
[2025-07-16 18:41:06,994: INFO:common:created directory at:artifacts/model_trainer]


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


[2025-07-16 18:41:14,800: WARNING:file_download:Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`]


Error while downloading from https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/pytorch_model.bin: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2536)
Trying to resume download...


[2025-07-16 19:03:01,921: WARNING:file_download:Error while downloading from https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/pytorch_model.bin: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2536)
Trying to resume download...]


KeyboardInterrupt: 